In [1]:
# Import all the required libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import the random forest classifier - This is a classification problem and I think this is the best model for prediction
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [3]:
# Load the cleaned dataset and view the first few rows
df = pd.read_csv('data/cleaned_df_v2.csv')
df['date_recorded'] = pd.to_datetime(df['date_recorded'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     59400 non-null  int64         
 1   amount_tsh             59400 non-null  float64       
 2   date_recorded          59400 non-null  datetime64[ns]
 3   funder                 59400 non-null  object        
 4   gps_height             59400 non-null  int64         
 5   longitude              59400 non-null  float64       
 6   latitude               59400 non-null  float64       
 7   basin                  59400 non-null  object        
 8   region                 59400 non-null  object        
 9   district_code          59400 non-null  int64         
 10  lga                    59400 non-null  object        
 11  population             59400 non-null  int64         
 12  scheme_management      59400 non-null  object        
 13  c

In [4]:
# Defining X and y columns to use with our models
X = df.drop('status_group', axis=1)
#X.columns
y = df['status_group']

# Using train_test_split to split our dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(41580, 24) (17820, 24)
(41580,) (17820,)


In [5]:
rf = RandomForestClassifier(n_estimators= 100)
rf.fit(X_train, y_train)

TypeError: invalid type promotion

In [ ]:
df.info()